<a href="https://colab.research.google.com/github/aguscura/UBA-data-science/blob/main/spark_S29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**S29:** Calcule el percentil 95 de la cantidad de fans de los usuarios de forma distribuida.(2 PUNTOS)

In [19]:
!pip install pyspark
# !pip install -U -q PyDrive
# !apt update
# !apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
from tqdm import tqdm

In [21]:
!gdown "1owjSBHKLx0pJMdQYtKUIfiXqBV3lfCdS"

Downloading...
From: https://drive.google.com/uc?id=1owjSBHKLx0pJMdQYtKUIfiXqBV3lfCdS
To: /content/user.csv
100% 2.71G/2.71G [00:20<00:00, 132MB/s]


In [22]:
#Creamos el Spark Context
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Paso el csv a parquet. Por un tema de RAM solo me traigo algunas columnas

In [23]:
#Chequeo columnas del df
df = pd.read_csv("/content/user.csv")
df.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

In [24]:
#Paso a parquet. Acá solo tomo algunas columnas por el RAM
df = pd.read_csv("/content/user.csv", usecols=["user_id","name","fans"])
df.to_parquet("/content/user.parquet")

In [25]:
#Leemos archivo parquet
sqlContext = SQLContext(sc)
rdd = sqlContext.read.parquet('/content/user.parquet').rdd.repartition(2)


In [26]:
#Cacheo el rdd con la transformación de traer solo los fans
rddCached = rdd.map(lambda x: x.fans).cache()

In [27]:
#Cantidad total de datos
count = rddCached.count()
count

1987897

In [28]:
#Indice de los datos que me va a dar el percentil 95. (Uso ceil para redondear para arriba)
import math
fans_95 = math.ceil(0.95 * count)
fans_95

1888503

In [29]:
#Consulta final - Obtengo el percentil 95.
rddCached.takeOrdered(count, lambda x: x)[fans_95]

4

Rta --> El percentil 95 de la cantidad de fans de los usuarios es 4.